### Лабораторная работа №3. Реализация сверточной нейронной сети

Данные: В работе предлагается использовать набор данных notMNIST, который состоит из изображений размерностью 28×28 первых 10 букв латинского алфавита (A … J, соответственно). Обучающая выборка содержит порядка 500 тыс. изображений, а тестовая – около 19 тыс.  

Данные можно скачать по ссылке:
https://commondatastorage.googleapis.com/books1000/notMNIST_large.tar.gz (большой набор данных);  
https://commondatastorage.googleapis.com/books1000/notMNIST_small.tar.gz (маленький набор данных);  

Описание данных на английском языке доступно по ссылке:  
http://yaroslavvb.blogspot.sg/2011/09/notmnist-dataset.html

Задание 1.  
Реализуйте нейронную сеть с двумя сверточными слоями, и одним полносвязным с нейронами с кусочно-линейной функцией активации. Какова точность построенное модели?

Задание 2.  
Замените один из сверточных слоев на слой, реализующий операцию пулинга (Pooling) с функцией максимума или среднего. Как это повлияло на точность классификатора?

Задание 3.  
Реализуйте классическую архитектуру сверточных сетей LeNet-5 (http://yann.lecun.com/exdb/lenet/).

Задание 4.  
Сравните максимальные точности моделей, построенных в лабораторных работах 1-3. Как можно объяснить полученные различия?

Результат выполнения заданий опишите в отчете.


In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from util import extract_dataset, download_dataset, read_dataset

In [3]:
dataset_url = "https://commondatastorage.googleapis.com/books1000/notMNIST_large.tar.gz"
dataset_path = extract_dataset(download_dataset(dataset_url))

In [4]:
img_height, img_width = 28, 28
known_classes=["A","B","C","D","E","F","G","H","I","J"]
X, y = read_dataset(dataset_path, known_classes, img_height, img_width)

In [5]:
X = X.reshape((-1, img_height, img_width, 1))
X.shape

(529114, 28, 28, 1)

In [6]:
y = y.T
y.shape

(529114, 1)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)
epochs_num = 100
batch_size = 512

Базовая сеть

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu', input_shape=(img_height, img_width, 1)),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),        
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
model.fit(X_train, Y_train, epochs=epochs_num, batch_size=batch_size)
model.evaluate(X_test, Y_test)

Train on 423291 samples
Epoch 1/100
423291/423291 [==============================] - 7s 16us/sample - loss: 2.2452 - accuracy: 0.1535
Epoch 2/100
423291/423291 [==============================] - 5s 11us/sample - loss: 2.1883 - accuracy: 0.1751
Epoch 3/100
423291/423291 [==============================] - 5s 11us/sample - loss: 2.1611 - accuracy: 0.1872
Epoch 4/100
423291/423291 [==============================] - 5s 11us/sample - loss: 2.1413 - accuracy: 0.1993
Epoch 5/100
423291/423291 [==============================] - 5s 11us/sample - loss: 2.1246 - accuracy: 0.2092
Epoch 6/100
423291/423291 [==============================] - 5s 11us/sample - loss: 2.1107 - accuracy: 0.2165
Epoch 7/100
423291/423291 [==============================] - 5s 11us/sample - loss: 2.0996 - accuracy: 0.2218
Epoch 8/100
423291/423291 [==============================] - 5s 11us/sample - loss: 2.0889 - accuracy: 0.2277
Epoch 9/100
423291/423291 [==============================] - 5s 11us/sample - loss: 2.0792 - acc

423291/423291 [==============================] - 5s 11us/sample - loss: 1.8606 - accuracy: 0.3161
Epoch 75/100
423291/423291 [==============================] - 5s 11us/sample - loss: 1.8593 - accuracy: 0.3164
Epoch 76/100
423291/423291 [==============================] - 5s 11us/sample - loss: 1.8590 - accuracy: 0.3166
Epoch 77/100
423291/423291 [==============================] - 5s 11us/sample - loss: 1.8565 - accuracy: 0.3175
Epoch 78/100
423291/423291 [==============================] - 5s 11us/sample - loss: 1.8563 - accuracy: 0.3176
Epoch 79/100
423291/423291 [==============================] - 5s 12us/sample - loss: 1.8554 - accuracy: 0.3178
Epoch 80/100
423291/423291 [==============================] - 5s 11us/sample - loss: 1.8547 - accuracy: 0.3178
Epoch 81/100
423291/423291 [==============================] - 5s 11us/sample - loss: 1.8541 - accuracy: 0.3178
Epoch 82/100
423291/423291 [==============================] - 5s 11us/sample - loss: 1.8530 - accuracy: 0.3185
Epoch 83/100
4

[2.842839310379492, 0.16198747]

Заменим один блок сверток на MaxPooling

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu', input_shape=(img_height, img_width, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),        
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
model.fit(X_train, Y_train, epochs=epochs_num, batch_size=batch_size)
model.evaluate(X_test, Y_test)

Train on 423291 samples
Epoch 1/100
423291/423291 [==============================] - 3s 8us/sample - loss: 2.2505 - accuracy: 0.1504
Epoch 2/100
423291/423291 [==============================] - 3s 7us/sample - loss: 2.2060 - accuracy: 0.1674
Epoch 3/100
423291/423291 [==============================] - 3s 7us/sample - loss: 2.1870 - accuracy: 0.1728
Epoch 4/100
423291/423291 [==============================] - 3s 8us/sample - loss: 2.1781 - accuracy: 0.1770
Epoch 5/100
423291/423291 [==============================] - 3s 7us/sample - loss: 2.1702 - accuracy: 0.1818
Epoch 6/100
423291/423291 [==============================] - 3s 7us/sample - loss: 2.1632 - accuracy: 0.1856
Epoch 7/100
423291/423291 [==============================] - 3s 7us/sample - loss: 2.1579 - accuracy: 0.1894
Epoch 8/100
423291/423291 [==============================] - 3s 8us/sample - loss: 2.1531 - accuracy: 0.1920
Epoch 9/100
423291/423291 [==============================] - 3s 7us/sample - loss: 2.1477 - accuracy: 0.

423291/423291 [==============================] - 3s 7us/sample - loss: 2.0472 - accuracy: 0.2316
Epoch 76/100
423291/423291 [==============================] - 3s 8us/sample - loss: 2.0458 - accuracy: 0.2322
Epoch 77/100
423291/423291 [==============================] - 3s 7us/sample - loss: 2.0450 - accuracy: 0.2324
Epoch 78/100
423291/423291 [==============================] - 3s 7us/sample - loss: 2.0447 - accuracy: 0.2314
Epoch 79/100
423291/423291 [==============================] - 3s 6us/sample - loss: 2.0446 - accuracy: 0.2320
Epoch 80/100
423291/423291 [==============================] - 3s 7us/sample - loss: 2.0436 - accuracy: 0.2319
Epoch 81/100
423291/423291 [==============================] - 3s 7us/sample - loss: 2.0426 - accuracy: 0.2325
Epoch 82/100
423291/423291 [==============================] - 3s 7us/sample - loss: 2.0421 - accuracy: 0.2329
Epoch 83/100
423291/423291 [==============================] - 3s 7us/sample - loss: 2.0421 - accuracy: 0.2327
Epoch 84/100
423291/423

[2.1151170901322702, 0.21063474]

Le-Net 5

In [10]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(img_height, img_width, 1)),
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2),
        tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu'),
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2),
        tf.keras.layers.Flatten(),        
        tf.keras.layers.Dense(120, activation='relu'),
        tf.keras.layers.Dense(84, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
model.fit(X_train, Y_train, epochs=epochs_num, batch_size=batch_size)
model.evaluate(X_test, Y_test)

Train on 423291 samples
Epoch 1/100
423291/423291 [==============================] - 4s 10us/sample - loss: 2.2401 - accuracy: 0.1559
Epoch 2/100
423291/423291 [==============================] - 4s 8us/sample - loss: 2.1954 - accuracy: 0.1714
Epoch 3/100
423291/423291 [==============================] - 4s 9us/sample - loss: 2.1692 - accuracy: 0.1832
Epoch 4/100
423291/423291 [==============================] - 4s 9us/sample - loss: 2.1539 - accuracy: 0.1884
Epoch 5/100
423291/423291 [==============================] - 4s 9us/sample - loss: 2.1416 - accuracy: 0.1938
Epoch 6/100
423291/423291 [==============================] - 4s 9us/sample - loss: 2.1349 - accuracy: 0.1969
Epoch 7/100
423291/423291 [==============================] - 4s 9us/sample - loss: 2.1294 - accuracy: 0.1997
Epoch 8/100
423291/423291 [==============================] - 4s 9us/sample - loss: 2.1256 - accuracy: 0.2012
Epoch 9/100
423291/423291 [==============================] - 4s 9us/sample - loss: 2.1215 - accuracy: 0

423291/423291 [==============================] - 4s 9us/sample - loss: 1.9573 - accuracy: 0.2630
Epoch 76/100
423291/423291 [==============================] - 4s 9us/sample - loss: 1.9552 - accuracy: 0.2642
Epoch 77/100
423291/423291 [==============================] - 4s 9us/sample - loss: 1.9578 - accuracy: 0.2633
Epoch 78/100
423291/423291 [==============================] - 4s 8us/sample - loss: 1.9554 - accuracy: 0.2643
Epoch 79/100
423291/423291 [==============================] - 4s 9us/sample - loss: 1.9552 - accuracy: 0.2642
Epoch 80/100
423291/423291 [==============================] - 4s 9us/sample - loss: 1.9578 - accuracy: 0.2633
Epoch 81/100
423291/423291 [==============================] - 4s 9us/sample - loss: 1.9537 - accuracy: 0.2647
Epoch 82/100
423291/423291 [==============================] - 4s 9us/sample - loss: 1.9551 - accuracy: 0.2645
Epoch 83/100
423291/423291 [==============================] - 4s 9us/sample - loss: 1.9535 - accuracy: 0.2646
Epoch 84/100
423291/423

[1.9901256653944939, 0.24648705]